In [6]:
import os
import click
import pandas as pd
import datetime
import traceback
from jqdatasdk import *
import cn_stock_holidays.data as shsz

class JQDataLoader():
    
    def __init__(self, date, data_dir):
        
        self.date = date
        self.date_str = self.date.strftime('%Y%m%d')
        self.data_dir = data_dir
    
    @staticmethod
    def authentication():
        username = '18140562236'
        password = 'Sinoalgo123'
        auth(username, password)
    
    def load_market_data(self, ukeys):
        
        data_list = []
        date_folder_str = self.date.strftime('%Y/%m/%d')
        for ukey in ukeys:
            marketType = ukey.rsplit('.')[-1]
            dir_name = f'{self.data_dir}/JQMarketData/STOCK/{marketType}/{date_folder_str}'
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            
            print(f'loading tick data for {ukey}')
            try:
                df = get_ticks(ukey, start_dt = self.date, end_dt = (self.date + datetime.timedelta(days = 1)),skip=False)
                df = self.format_ticket_data(df)
                df.to_csv(os.path.join(dir_name, ukey) + '.gz', index = False, compression = 'gzip')
            except:
                print(f'failed to load tick data for {ukey}')
    
    @staticmethod
    def format_ticket_data(data):
        
        data['time'] = data['time'].apply(lambda x: str(x).replace("-", "").replace(":", "").replace(" ", "") + "000")
        priceCols = ['current', 'high', 'low'] + [ x for x in data.columns if x[-1] == 'p']
        for col in priceCols:
            data[col] = data[col].apply( lambda x: int(round(x*10000)) )
        
        volCols = ['volume'] + [x for x in data.columns if x[-1] == 'v']
        for col in volCols:
            data[col] = data[col].apply( lambda x: int(x) )
        
        return data
    
    def run(self):
        
        
        security_df = get_all_securities(types = ['stock'], date = self.date).reset_index()
        security_df.rename({'index' : 'code'}, axis = 1, inplace = True)
        
        ukeys =['002703.XSHE','000682.XSHE']


        close_df = get_price(ukeys, start_date = self.date, end_date = self.date, fields = ['open', 'close', 'high', 'low', 'volume', 'money', 'high_limit', 'low_limit', 'factor'])
        
        # add check_is_st information
        extras = get_extras('is_st', ukeys, start_date = self.date, end_date = self.date)
        
        def check_is_st(code, time):
            return 1 if extras.loc[time, code] else 0
        
        close_df['is_st'] = close_df.apply(lambda x: check_is_st(x['code'], x['time']), axis = 1)
        close_df = close_df.drop_duplicates(['code'])
        
        security_df = security_df.merge(close_df[['code', 'close', 'high_limit', 'low_limit', 'is_st', 'factor']], on = ['code'], how = 'left')
        
        self.load_market_data(ukeys)
        date_folder_str = self.date.strftime('%Y/%m/%d')

        
        os.system(f'chmod 777 -R {self.data_dir}/JQMarketData/STOCK/XSHE/{date_folder_str}')
        os.system(f'chmod 777 -R {self.data_dir}/JQMarketData/STOCK/XSHG/{date_folder_str}')


def main(start, end, today=False):
    if today:

        today_date = datetime.date.today()
        while not shsz.is_trading_day(today_date) or datetime.datetime.now() < datetime.datetime(year=today_date.year, month=today_date.month, day=today_date.day, hour=15):
            today_date -= datetime.timedelta(days=1)
        prev_date = today_date - datetime.timedelta(days=1)
        while not shsz.is_trading_day(prev_date):# or datetime.datetime.now() < datetime.datetime(year=today_date.year, month=today_date.month, day=today_date.day, hour=15):
            prev_date -= datetime.timedelta(days=1)
        dates = [today_date, prev_date]
    else:
        dates = [i.date() for i in pd.date_range(start, end, freq = 'D')]
        dates = [date for date in dates if shsz.is_trading_day(date)]
    
    JQDataLoader.authentication()
    for date in dates:
        print(f'Running {date}')
        try:
            JQDataLoader(date, '/volume1/home/rzhu').run()
        except Exception as e:
            print(f'failed to load data for {date}')
            print(traceback.format_exc())

if __name__ == '__main__':
    start = '2021-05-31'
    end = '2022-03-24'
    main(start,end)

Running 2021-05-31
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-01
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-02
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-03
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-04
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-07
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-08
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-09
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-10
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-11
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-15
loading tick data for 002703.XSHE
loading tick data for 000682.XSHE
Running 2021-06-16
loading tick data for 00

In [1]:
import os
import click
import pandas as pd
import datetime
import traceback
from jqdatasdk import *
import cn_stock_holidays.data as shsz

class JQDataLoader():
    
    def __init__(self, date, data_dir, ukeys):
        
        self.date = date
        self.date_str = self.date.strftime('%Y%m%d')
        self.data_dir = data_dir
        self.ukeys = ukeys
    
    @staticmethod
    def authentication():
        username = '18140562236'
        password = 'Sinoalgo123'
        auth(username, password)
    
    def load_market_data(self):
        
        data_list = []
        date_folder_str = self.date.strftime('%Y/%m/%d')
        for ukey in self.ukeys:
            marketType = ukey.rsplit('.')[-1]
            dir_name = f'{self.data_dir}/JQMarketData/STOCK/{marketType}/{date_folder_str}'
            if not os.path.exists(dir_name):
                os.makedirs(dir_name)
            
            print(f'loading tick data for {ukey}')
            try:
                df = get_ticks(ukey, start_dt=self.date, end_dt=(self.date + datetime.timedelta(days=1)), skip=False)
                df = self.format_ticket_data(df)
                df.to_csv(os.path.join(dir_name, ukey) + '.gz', index=False, compression='gzip')
            except:
                print(f'failed to load tick data for {ukey}')
    
    @staticmethod
    def format_ticket_data(data):
        
        data['time'] = data['time'].apply(lambda x: str(x).replace("-", "").replace(":", "").replace(" ", "") + "000")
        priceCols = ['current', 'high', 'low'] + [x for x in data.columns if x[-1] == 'p']
        for col in priceCols:
            data[col] = data[col].apply(lambda x: int(round(x * 10000)))
        
        volCols = ['volume'] + [x for x in data.columns if x[-1] == 'v']
        for col in volCols:
            data[col] = data[col].apply(lambda x: int(x))
        
        return data
    
    def run(self):
        
        security_df = get_all_securities(types=['stock'], date=self.date).reset_index()
        security_df.rename({'index': 'code'}, axis=1, inplace=True)
        
        close_df = get_price(self.ukeys, start_date=self.date, end_date=self.date,
                             fields=['open', 'close', 'high', 'low', 'volume', 'money', 'high_limit', 'low_limit', 'factor'])
        
        # add check_is_st information
        extras = get_extras('is_st', self.ukeys, start_date=self.date, end_date=self.date)
        
        def check_is_st(code, time):
            return 1 if extras.loc[time, code] else 0
        
        close_df['is_st'] = close_df.apply(lambda x: check_is_st(x['code'], x['time']), axis=1)
        close_df = close_df.drop_duplicates(['code'])
        
        security_df = security_df.merge(
            close_df[['code', 'close', 'high_limit', 'low_limit', 'is_st', 'factor']], on=['code'], how='left')
        
        self.load_market_data()
        date_folder_str = self.date.strftime('%Y/%m/%d')
        
        os.system(f'chmod 777 -R {self.data_dir}/JQMarketData/STOCK/XSHE/{date_folder_str}')
        os.system(f'chmod 777 -R {self.data_dir}/JQMarketData/STOCK/XSHG/{date_folder_str}')


def main(start, end, today=False, ukeys=None):
    if today:
        today_date = datetime.date.today()
        while not shsz.is_trading_day(today_date) or datetime.datetime.now() < datetime.datetime(
                year=today_date.year, month=today_date.month, day=today_date.day, hour=15):
            today_date -= datetime.timedelta(days=1)
        prev_date = today_date - datetime.timedelta(days=1)
        while not shsz.is_trading_day(prev_date):
            prev_date -= datetime.timedelta(days=1)
        dates = [today_date, prev_date]
    else:
        dates = [i.date() for i in pd.date_range(start, end, freq='D')]
        dates = [date for date in dates if shsz.is_trading_day(date)]

    JQDataLoader.authentication()
    if ukeys is None:
        ukeys = ['002703.XSHE', '000682.XSHE']  # 设置ukeys的默认值
    for date in dates:
        print(f'Running {date}')
        try:
            JQDataLoader(date, '/volume1/home/rzhu', ukeys).run()
        except Exception as e:
            print(f'failed to load data for {date}')
            print(traceback.format_exc())

if __name__ == '__main__':
    start = '2021-05-31'
    end = '2021-05-31'
    main(start, end, ukeys=['000001.XSHE'])


auth success 
Running 2021-05-31
loading tick data for 000001.XSHE
